In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
import _pickle as cPickle
import numpy as np
import os

CIFAR_DIR = "/content/drive/My Drive/Colab Notebooks/cifar-10-batches-py"
print(os.listdir(CIFAR_DIR)) 

['readme.html', 'batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_5', 'test_batch', 'data_batch_4', 'data_batch_3', '.ipynb_checkpoints']


In [3]:
def load_data(filename):
    """read data from data file."""
    
    # 注： 在load数据时，通过添加encoding='bytes'，解决'ascii' codec can't decode byte 0x8b in position 6: ordinal not in range(128)
    with open(filename, 'rb') as f:
        data = cPickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']

# tensorflow.Dataset  同样可以实现CifarData的效果
class CifarData:
    def __init__(self, filenames, need_shuffle): # need_shuffle: 指是否需要打乱样本的顺序，越无序，训练的结果越好
        all_data = []
        all_labels = []
        for filename in filenames:
            datas, labels = load_data(filename)
            for data, label in zip(datas, labels):
                if label in [0,1]: # 因为我们要做二分类，故只筛选出 label是0，1的数据
                    all_data.append(data)
                    all_labels.append(label)
        
        self._data = np.vstack(all_data)
         
        # 对数据做归一化处理，像素值都是在0-255之间，故做如下处理，将其归一化在（-1，1）之间 # 提高训练的准确率
        # 对比，没有self._data = self._data / 127.5 - 1 和有此语句的准确率
        # 因为像素值是在 0——255之间，所以，/127.5，其值在0-2之间，再减去1，则确保元素的值在（-1，1）之间。
        # 因为 sigmoid函数若数据偏向一方，梯度会消失，归一化之后将不再会出现这种问题
        self._data = self._data / 127.5 - 1 
        self._labels = np.hstack(all_labels)
        
        
        # print("self._data.shape")
        # print(self._data.shape)
        # print("self._labels.shape")
        # print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0  # self._indicator 用来分batch的时候用
        
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self):
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        
        self._indicator = end_indicator
        return batch_data, batch_labels
    
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

batch_data, batch_labels = train_data.next_batch(500)
# print('batch_data')
# print(batch_data)
# print('batch_labels')
# print(batch_labels)

self._data.shape
(10000, 3072)
self._labels.shape
(10000,)
self._data.shape
(2000, 3072)
self._labels.shape
(2000,)


In [0]:
x = tf.placeholder(tf.float32, [None, 3072]) # 32*32*3 = 3072
y = tf.placeholder(tf.int64, [None])

# tf.random_normal_initializer: Initializer that generates tensors with a normal distribution.
# currently， the shape of w is [3072 * 1]
w = tf.get_variable('w', [x.get_shape()[-1], 1], initializer = tf.random_normal_initializer(0, 1)) 

# tf.constant_initializer: Initializer that generates tensors with constant values.
# currently， the b is a constant value 0.
b = tf.get_variable('b', [1], initializer = tf.constant_initializer(0.0))  # b被初始化为0

# tf.matmul: Multiplies matrix a by matrix b, producing a * b.
y_ = tf.matmul(x, w) + b   #  [None, 3027] * [3027, 1] = [None, 1]
p_y_1 = tf.nn.sigmoid(y_)

# 由于变量y之前定义的shape是None，故需要对y进行reshape为[None, 1]
y_reshaped = tf.reshape(y, (-1, 1))

# 由于后续，损失函数，真实值y和预测值p_y_1间要进行运算，故需保持数据类型一样
y_reshaped_float = tf.cast(y_reshaped, tf.float32)  # tf.cast: 类型转换 Casts a tensor to a new type.

# tf.square(y_reshaped_float - p_y_1), 真实值和预测值之间差值的平方
# tf.reduce_mean: Computes the mean of elements across dimensions of a tensor.
# loss 即为平方差损失
loss = tf.reduce_mean(tf.square(y_reshaped_float - p_y_1))

# bool
predict = p_y_1 > 0.5

#[1, 0, 1, 1, 1, 0, 0, 0]
correct_prediction = tf.equal(tf.cast(predict, tf.int64), y_reshaped)

# 准确率 
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

# tf,name_scope: A context manager for use when defining a Python op.
# tf.train.AdamOptimizer: Optimizer that implements the Adam algorithm.
# ? Adam algorithm

# tf.train.AdamOptimizer.minimize
# Add operations to minimize loss by updating var_list.
# This method simply combines calls compute_gradients() and apply_gradients(). 
# If you want to process the gradient before applying them call compute_gradients() and apply_gradients() 
# explicitly instead of using this function
# 定义梯度下降的方法
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss) # 1e-3为初始的running rate（学习率）
    

In [6]:
init = tf.global_variables_initializer()
batch_size = 500
train_steps = 5000
test_steps = 4

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps): 
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op],
            feed_dict = {
                x: batch_data,
                y: batch_labels})
        
        if i % 50 == 0:
            print('[Train] step: %d, loss: %4.5f, acc: %4.5f'\
                 % (i, loss_val, acc_val))
        
        # 在测试集上评测的代码
        if i % 50 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data,
                        y: test_batch_labels})
                
                all_test_acc_val.append(test_acc_val)
            
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f' % (i, test_acc))

[Train] step: 0, loss: 0.62435, acc: 0.36400
self._data.shape
(2000, 3072)
self._labels.shape
(2000,)
[Test] Step: 0, acc: 0.33500
[Train] step: 50, loss: 0.43463, acc: 0.55200
self._data.shape
(2000, 3072)
self._labels.shape
(2000,)
[Test] Step: 50, acc: 0.53400
[Train] step: 100, loss: 0.35424, acc: 0.63200
self._data.shape
(2000, 3072)
self._labels.shape
(2000,)
[Test] Step: 100, acc: 0.66600
[Train] step: 150, loss: 0.32315, acc: 0.67200
self._data.shape
(2000, 3072)
self._labels.shape
(2000,)
[Test] Step: 150, acc: 0.69950
[Train] step: 200, loss: 0.28822, acc: 0.70200
self._data.shape
(2000, 3072)
self._labels.shape
(2000,)
[Test] Step: 200, acc: 0.72200
[Train] step: 250, loss: 0.28452, acc: 0.70400
self._data.shape
(2000, 3072)
self._labels.shape
(2000,)
[Test] Step: 250, acc: 0.73750
[Train] step: 300, loss: 0.28519, acc: 0.70600
self._data.shape
(2000, 3072)
self._labels.shape
(2000,)
[Test] Step: 300, acc: 0.74550
[Train] step: 350, loss: 0.24936, acc: 0.74600
self._data.sha